# Hello-World DSPy Agent

> A minimal DSPy agent implementation to echo or compute simple inputs.

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import dspy
import os

# Configure a default LLM - using gpt-4o-mini as specified in the plan
# Note: In real usage, you'd set OPENAI_API_KEY in your environment
default_lm = dspy.LM("openai/gpt-4o-mini")
dspy.configure(lm=default_lm)

In [ ]:
#| export
class Echo(dspy.Module):
    """Echoes the input message back."""
    signature = dspy.Signature(
        "message -> str",
        "Echoes the input message back."
    )
    def forward(self, message: str) -> str:
        return message

In [ ]:
#| export
from typing import Dict, Any, List, Optional
from cogitarelink_dspy.wrappers import get_tools, group_tools_by_layer

def make_hello_agent(llm=None):
    """Create a simple DSPy agent with dynamic tool loading and LLM capabilities.
    
    Args:
        llm: A DSPy language model. If None, uses the default configured LM.
    
    Returns:
        A HelloAgent instance that can process messages using all available tools and LLM.
    """
    # Use provided LLM or the default one
    lm = llm or dspy.settings.lm
    
    # Create a simple signature for LLM-based response generation
    signature = dspy.Signature(
        "message -> response",
        "Generate a friendly response to the input message."
    )
    
    # Create an Agent class that can use all tools and LLM
    class HelloAgent(dspy.Module):
        """A Semantic Web agent that understands layer architecture and uses the appropriate tools.
        
        This agent dynamically loads all available tools from the component registry
        and organizes them by semantic layer.
        """
        # Define signature with instructions to avoid DSPy error
        signature = dspy.Signature(
            "message -> echo_result, llm_response, layer_used",
            "Generate a response using tools and identify the semantic layer."
        )
        
        def __init__(self):
            super().__init__()
            # Initialize predictor for LLM responses
            self.predictor = dspy.ChainOfThought(signature)
            
            try:
                # Try to load tools, but have a fallback for testing
                self.tools = get_tools()
                self.tools_by_layer = group_tools_by_layer(self.tools)
            except Exception as e:
                # Fallback for notebook testing
                print(f"Warning: Failed to load tools: {e}")
                self.tools = []
                self.tools_by_layer = {"Utility": []}
            
        def forward(self, message: str) -> Dict[str, Any]:
            """Process a message using tools and LLM.
            
            Args:
                message: The input message to process
                
            Returns:
                Dict with echo_result, llm_response, and other fields
            """
            # First, try to match the message to a semantic layer
            # For simplicity here, we're just using a simple keyword-based approach
            # In a real system, this would be a more sophisticated matching system
            layer_used = "Utility"  # Default layer
            layer_keywords = {
                "context": "Context",
                "json-ld": "Context",
                "namespace": "Context",
                "ontology": "Ontology",
                "vocabulary": "Ontology",
                "rule": "Rules",
                "validate": "Rules",
                "triple": "Instances",
                "instances": "Instances",
                "graph": "Instances",
                "signature": "Verification",
                "verify": "Verification",
                # Catch noun form as well
                "verification": "Verification"
            }
            
            message_lower = message.lower()
            for keyword, layer in layer_keywords.items():
                if keyword in message_lower:
                    layer_used = layer
                    break
            
            # Get tools for the identified layer
            layer_tools = self.tools_by_layer.get(layer_used, [])
            
            # For demo purposes, we'll just use the first tool in the layer if available
            # In a real system, this would be a more intelligent tool selection
            tool_result = "No tool available for this layer"
            tool_used = "None"
            
            if layer_tools:
                # Get the first tool and instantiate it
                tool_class = layer_tools[0]
                tool = tool_class()
                tool_used = tool_class.__name__
                
                # For demo purposes, we just acknowledge we would call the tool
                # In a real system with actual implementations, we would call the tool
                tool_result = f"Would call {tool_used} from layer {layer_used}"
            
            # Get LLM response
            llm_result = self.predictor(message=message)
            
            # Return the results
            return {
                "echo_result": message,  # Simple echo
                "llm_response": llm_result.response,
                "layer_used": layer_used,
                "tool_used": tool_used,
                "tool_result": tool_result,
                "reasoning": getattr(llm_result, "reasoning", None)
            }
            
    # Return a new instance of the HelloAgent
    return HelloAgent()

In [ ]:
# Demo the Hello-World agent with dynamic tool loading capabilities
try:
    agent = make_hello_agent()
    
    # Test with a simple message
    message = "Hello, DSPy! What can you do with Semantic Web data?"
    print(f"Testing with message: \"{message}\"")
    result = agent(message)
    
    print(f"\nLayer identified: {result.get('layer_used', 'Unknown')}")
    print(f"Tool selected: {result.get('tool_used', 'None')}")
    print(f"\nLLM response: {result.get('llm_response', 'No response')}")
except Exception as e:
    print(f"Error testing agent: {e}")
    print("This is expected during notebook testing without a real LLM connection.")

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()